##### 필요 라이브러리 import

In [1]:
import os
import sys
import requests
import re
import time
import json
import pandas as pd
import csv
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings('ignore')

##### 초기 설정

In [2]:
extracts = re.compile('[^ 가-힣|a-z|A-Z|0-9|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')

nowDatetime = datetime.now().strftime('%Y-%m-%d') # 날짜 표기를 위함

##### csv 파일 생성

In [3]:
# 서울
# data_url = open('./data/goodchoice_seoul[' + nowDatetime +'].csv', 'w', encoding='utf8') # url 저장 관련 csv파일
# data = open('./data/info_seoul[' + nowDatetime +'].csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일
# data_r = open('./reviews/reviews_seoul[' + nowDatetime +'].csv', 'w', encoding='utf8') # 리뷰 저장관련 csv 파일

# 부산
data_url = open('./data/goodchoice_busan[' + nowDatetime +'].csv', 'w', encoding='utf8') # url 저장 관련 csv파일
data = open('./data/info_busan[' + nowDatetime +'].csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일
data_r = open('./reviews/reviews_busan[' + nowDatetime +'].csv', 'w', encoding='utf8') # 리뷰 저장관련 csv 파일

##### Chrome 파일 열고 url 입력받음

In [4]:
driver = webdriver.Chrome("chromedriver.exe")
# driver.maximize_window() #전체 화면
# Url = 'https://www.goodchoice.kr/product/search/1/7052' #서울 역삼, 강남, 논현
Url = 'https://www.goodchoice.kr/product/search/1/7044' #부산 광안리, 수영, 민락
driver.get(Url)

In [5]:
urls = driver.find_element(By.ID, "poduct_list_area")
urlList = urls.find_elements(By.TAG_NAME, 'li')

for li in urlList:
    aTag = li.find_element(By.TAG_NAME, 'a')
    href = aTag.get_attribute('href')
    data_url.write(href)
    data_url.write('\n')

data_url.close()    

##### data.csv 파일 구분 입력

In [6]:
data.write('"name"')
data.write(', ')
data.write('"basic_info"')
data.write(', ')
data.write('"cost_info"')
data.write(', ')
data.write('"avg_stars"')
data.write('\n')

1

##### 사이트 주소를 하나씩 읽기 위해 f를 열고 rdr 파일에 reader로 csv파일을 읽음

In [7]:
# 서울
# f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_seoul[' + nowDatetime +'].csv', 'r', encoding='utf8') # 노트북
# f = open('D:/webcc/data/goodchoice_seoul[' + nowDatetime +'].csv', 'r', encoding='utf8') #622 컴퓨터

# rdr = csv.reader(f)

In [8]:
# 부산
f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_busan[' + nowDatetime +'].csv', 'r', encoding='utf8') #노트북
# f = open('D:/webcc/data/goodchoice_busan[' + nowDatetime +'].csv', 'r', encoding='utf8') #622 컴퓨터

rdr = csv.reader(f)

##### 기본 정보, 요금 정보 및 평균 평점 입력 받아 info_data.csv파일에 입력

In [9]:
for line in rdr:
    addr = line[0]
    # print(addr)
    rs = requests.get(addr)

    rs.encoding = None
    html = rs.text
    soup = BeautifulSoup(html, 'html.parser')

    try:
        tarname = soup.select('#content > div.top > div.right')
        rname = soup.find("div", {"class" : "info"})
        name = rname.find("h2")
        name = name.text

        tarbasic_info = soup.select('#content > article.detail_info > section.default_info')
        basic_info = ''
        for text in tarbasic_info:
            basic_info += text.getText()
        
        basic_info = extracts.sub('', basic_info)
        basic_info = re.sub(' +', ' ', basic_info)
        basic_info = '"'+basic_info+'"'

        tarcost_info = soup.select('#content > article.detail_info.on > section.table_wrap > div:nth-child(2) > table:nth-child(2)')
        rcost_info = soup.find("table", {"class" : "table_type_02"})
        cost_info = rcost_info.find("span")
        cost_info = cost_info.text

        tarstar_info = soup.select('#content > div.top > div.right > div.info')
        rstar_info = soup.find("div", {"class" : "score_cnt"})
        avg_star_info = rstar_info.find("span").text[0:4]

        data.write(name)
        data.write(', ')
        data.write(basic_info)
        data.write(', ')
        data.write(cost_info)
        data.write(', ')
        data.write(avg_star_info)
        data.write('\n')

    except:
        continue

f.close()
data.close()
driver.close()

KeyboardInterrupt: 

##### review.csv 파일 구분 입력

In [ ]:
data_r.write('"review"')
data_r.write('\n')

1

##### data_url.csv 파일에서 url을 받기 위해 csv파일을 읽을 수 있게 받음

In [ ]:
# 서울
# d = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_seoul[' + nowDatetime +'].csv', 'r', encoding='utf8') # 노트북
# d = open('D:/webcc/data/goodchoice_seoul[' + nowDatetime +'].csv', 'r', encoding='utf8') # 622 컴퓨터

# 부산 
d = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_busan[' + nowDatetime +'].csv', 'r', encoding='utf8') # 노트북
# d = open('D:/webcc/data/goodchoice_busan[' + nowDatetime +'].csv', 'r', encoding='utf8') # 622 컴퓨터

rdd = csv.reader(d)    

##### url을 한줄씩 받아와 한 url당 15개의 리뷰를 review.csv에 추가 입력

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")

for line in rdd:
   for i in range(1, 11):
      add = line[0]

      driver.get(f'add+'#{}').format(i)
      html2 = driver.page_source
      
    #   data_r = open('D:/webcc/reviews/reviews_seoul[' + nowDatetime +'].csv', 'a', encoding='utf8') #서울
    #   data_r = open('D:/webcc/reviews/reviews_busan[' + nowDatetime +'].csv', 'a', encoding='utf8') #부산
      data_r = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/reviews/reviews_busan[' + nowDatetime +'].csv', 'a', encoding='utf8')
      soups = BeautifulSoup(html2, 'html.parser')
      reviews = soups.find_all('div',class_= 'txt')

      for review in reviews:
          review_text = str(review)
          data_r.write(review_text)
          data_r.write('\n')

      data_r.write('----------------------')
      data_r.write('\n')
      data_r.close()  
    
 
d.close()
driver.close()